1. Preprocess
2. Data Loader
3. Model define

# LSTM

* https://github.com/SheezaShabbir/Time-series-Analysis-using-LSTM-RNN-and-GRU/blob/main/Pytorch_LSTMs%2CRNN%2CGRU_for_time_series_data.ipynb
* https://github.com/vincrichard/LSTM-AutoEncoder-Unsupervised-Anomaly-Detection/blob/master/DataChallengeReport_VincentRichard.ipynb
* https://github.com/yakhyo/pytorch-tutorials/blob/main/tutorials/03-intermediate/04-lstm-network/main.py
* https://github.com/yakhyo/pytorch-tutorials/blob/main/tutorials/03-intermediate/05-var-auto-encode/main.py
* https://github.com/vincrichard/LSTM-AutoEncoder-Unsupervised-Anomaly-Detection/blob/master/DataChallengeReport_VincentRichard.ipynb

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import pandas as pd
import torch.nn as nn
from torchsummary import summary
import torch.optim as optim
from tqdm.auto import tqdm

## 1. Preprocessing

In [52]:
df = pd.read_csv('./training_spiral.csv')
feature_list = ['Fx','Fy','Fz','Mx','My']
TIMESTEP = 50
BATCH_SIZE = 32

In [54]:
df.shape

(357823, 20)

In [57]:
df = df.iloc[:50000]

In [58]:
df.shape

(50000, 20)

In [59]:
df.Case.value_counts()

False    50000
Name: Case, dtype: int64

In [60]:
# def to_sequence(data, timesteps=1):
#     n_features=data.shape[2]
#     seq = []
#     for i in range(len(data)-timesteps):
#         # takes a window of data of specified timesteps
#         temp = data[i:(i+timesteps)]
#         temp = temp.reshape(timesteps, n_features)
#         seq.append(temp)
        
#     return np.array(seq)

def to_sequence(data, timesteps=1):
    n_features=data.shape[2]
    x = []
    y = []
    for i in range(len(data)-timesteps):
        # takes a window of data of specified timesteps
        
        _x = data[i:(i+timesteps)]
        _x = _x.reshape(timesteps, n_features)
#         print(_x.shape)
        _y = data[i+timesteps]
        _y = _y.reshape(n_features)
#         print(_y.shape)
        x.append(_x)
        y.append(_y)

        
    return np.array(x), np.array(y)

In [61]:
df_train = df[feature_list]
window = 40
df_train = df_train.rolling(window).mean()
# due to the moving average we the first (window-1) rows become NaN so we remove them
df_train = df_train.loc[window-1:]
print(df_train.shape)
train = np.expand_dims(df_train, axis=1)
x_train, y_train = to_sequence(train, timesteps=TIMESTEP)
print(x_train.shape)
print(y_train.shape)

(49961, 5)
(49911, 50, 5)
(49911, 5)


In [62]:
x_train_torch = torch.Tensor(x_train)
y_train_torch = torch.Tensor(y_train)
print(x_train_torch.shape)
print(y_train_torch.shape)
train = TensorDataset(x_train_torch, y_train_torch)

torch.Size([49911, 50, 5])
torch.Size([49911, 5])


In [63]:
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)

In [64]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 50, 5])
Labels batch shape: torch.Size([32, 5])


In [67]:
# x_train_torch = torch.Tensor(x_train)
# print(x_train_torch.shape)
# torch.save(x_train_torch, 'train_seq.pt')

## 2. Data Loader

In [68]:
# class SensorData(Dataset):
#     def __init__(self, file_path, file_type):
        
#         if file_type == 'csv':
#             self.data = pd.read_csv(file_path, delimiter=',', nrows=None, header=None)
#         elif file_type == 'pt':
#             self.data = torch.load(file_path)
#         else:
#             raise ValueError('type value is wrong: ', file_type)
#         self.file_type = file_type
        
#     def __len__(self):
#         if self.file_type == 'csv':
#             return len(self.data)
        
#         if self.file_type == 'pt':
#             return self.data.shape[0]
    
#     def __getitem__(self, idx):
#         if self.file_type == 'pt':
#             return self.data[idx,:,:]

### 2.1. Load Data

In [69]:
# train_dataset = SensorData(file_path = './train_seq.pt', file_type='pt')
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
# samples, timestep, features = train_dataset.data.shape

In [70]:
print(f"Length of Train Loader {len(train_loader)} batches of {BATCH_SIZE}")
# print(f"Data is of shape:", train_dataset.data.shape)

Length of Train Loader 1560 batches of 32


## 3. Model Building

In [71]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using:", device)

Using: cpu


## 3.1. LSTM 
LSTM: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

1. `input, (h_0, c_0)`
    * input: `(N, L, Hin)`
    * h_0: tensor of shape `(num_layers, N,hidden_size)`, defaults to zeros 
    * c_0: tensor of shape `(num_layers, N, Hcell)`
2. `output, (h_n, c_n)`
    * output: `(N,L,hidden_size)`
    * h_n: containing final hidden state for each element in the sequence: `(num_layers, N, hidden_size)`
    * c_0:  `(num_layers, N, Hcell)`

In [72]:
class LSTMModel(nn.Module):
    """
    Attributes:
        input_size: number of expected features in X
        hidden_size: how many LSTM cells are there in each hidden layer
        num_layers: how many stacked LSTMs we want to use
        output_dim: LSTM output shape
    """
    def __init__(self, input_size, hidden_size, num_layers, output_dim, print_info):
        super(LSTMModel, self).__init__()
        self.print_info = print_info
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
                
        # LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # Fully Connected
        # Output_dim == number of features (1 pred per feature)
        self.fc = nn.Linear(hidden_size, output_dim)
        
    def _init_hidden(self, X):
        batch_size = X.size(0)
        device = X.device
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        
        return h0, c0
        
    def forward(self, X):
        """
        input (X): should be of shape (batch_size, seq_length, hidden_size)
        """
        # initialize hidden and cell states
        h0, c0 = self._init_hidden(X)
        # call lstm
        out, (hn, cn) = self.lstm(X, (h0.detach(), c0.detach()))
        # for plotting
        init_out = out
        # out: batch_size, seq_len, hidden_size
        # out(N, 28, 128)
        out = out[:,-1,:]
        out_reshaped = out
        # we want last timestep: out (N, 128)
        out = self.fc(out)
        # batch_size, output_dim
        
        if self.print_info:
            print('X shape:', X.shape)
            print(f"h0 shape: {h0.shape}, c0 shape: {c0.shape}")
            print('init_out shape:', init_out.shape)
            print('output_reshaped', out_reshaped.shape)
            print('out shape:', out.shape)
            
        return out
        

In [78]:
#  input_size: number of expected features in X
#  hidden_size: how many LSTM cells are there in each hidden layer
#  num_layers: how many stacked LSTMs we want to use

input_size = 5
hidden_size = 32
num_layers = 1
output_dim = 5

sequence_dim = TIMESTEP

# instatiate model
model = LSTMModel(input_size, hidden_size, num_layers, output_dim, print_info=False)
# moving model to the GPU 
model = model.to(device)
print(model)

LSTMModel(
  (lstm): LSTM(5, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=5, bias=True)
)


In [79]:
# testing if model works/
# model(train_dataset.data[:2,:,:])

In [80]:
# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)
# loss
criterion = torch.nn.MSELoss()

## 4. Training

In [81]:
print(f"Length of Train Loader {len(train_loader)} batches of {BATCH_SIZE}")

Length of Train Loader 1560 batches of 32


In [95]:
def train_step(model, train_loader, criterion, optimizer, device):
    # training mode
    model.train()
    
    train_loss = 0
    # we track the batch index 
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):

        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        yhat = model(x_batch)

        loss = criterion(y_batch, yhat)
        train_loss += loss.item()
        
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        
    train_loss = train_loss/ len(train_loader)
    return train_loss

In [99]:
def test_step(model, test_loader, criterion, device):
    model.eval()
    
    # set test loss
    test_loss = 0
    # turn inference
    with torch.inference_mode():
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):

            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            yhat = model(x_batch)

            loss = criterion(y_batch, yhat)
            test_loss += loss.item()
        
        test_loss = test_loss/ len(test_loader)
        return test_loss

In [102]:
def train(model, train_loader, test_loader, optimizer, criterion, epochs, device):
    results = {"train_loss": [],
              "test_loss": []}
    
    for epoch in tqdm(range(epochs)):
        train_loss = train_step(model, train_loader, criterion, optimizer, device)
        
        test_loss = test_step(model, test_loader, criterion, device)
        
        print(f"Epoch: {epoch} | Train loss: {train_loss} | Test loss: {test_loss}")
        
        # update results dicts
        results['train_loss'].append(train_loss)
        results['test_loss'].append(test_loss)
        
    return results

In [104]:
def plot_loss_curves(results):
    loss = results['train_loss']
    test_loss = results['test_loss']
    
    epochs = range(len(results['train_loss']))
    
    plt.figure()
    plt.plot(epochs, loss)
    plt.show()